In [86]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

In [87]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp]} for sp in scores})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], "truth"] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], "truth"] = 1
    return df

In [88]:
def get_metrics(df_res, splits=4, K=None, th=0.5):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
        df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
        df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 1
        df = df_res[[sp, "truth"]].dropna()
        t = df["truth"].sum() if K is None else K
        t_frac = t / df.shape[0]
        # df["pred"] = (df[sp].rank(ascending=False, method="min") <= t) * 1
        # df["pred"] = df[sp] >= t_frac
        df["pred"] = df[sp] >= th

        ret[sp] = {
            "precision": sklearn.metrics.precision_score(y_true=df["truth"], y_pred=df["pred"]),
            "recall": sklearn.metrics.recall_score(y_true=df["truth"], y_pred=df["pred"]),
            "f1": sklearn.metrics.f1_score(y_true=df["truth"], y_pred=df["pred"])
        }
    return ret

In [89]:
alg_list = ["fraudar", "rsd", "rev2"][:3]
data_list = ["alpha", "otc", "amazon", "epinions"][:2]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.0, 0.2, 0.4, 0.6, 0.8]

In [90]:
attack_res = {
    (d, a, b): get_results(data_name=d, alg=a, attack_name="sockfarm_attack", budget=b)
    for d in data_list for a in alg_list
    for b in budget_list
}

In [94]:
attack_res[("alpha", "rev2", 400.0)].to_csv("ar.csv")

In [98]:
th_dict = {
    "fraudar": 0.5,
    "rsd": 0.6,
    "rev2": 0.5,
}
dict_met = {
    (d, a, b): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b)], splits=4, K=None, th=th_dict[a]))
    for d, a, b in attack_res
#     for b in budget_list for f in frac_list
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
display(df_met)

0         1         2         3
alpha fraudar 100.0 precision  0.352201  0.352201  0.352201  0.352201
                    recall     0.375839  0.375839  0.375839  0.375839
                    f1         0.363636  0.363636  0.363636  0.363636
              200.0 precision  0.352201  0.352201  0.352201  0.352201
                    recall     0.244541  0.244541  0.244541  0.244541
...                                 ...       ...       ...       ...
otc   rev2    300.0 recall     0.111765  0.111765  0.111765  0.111765
                    f1         0.196382  0.196382  0.196382  0.196382
              400.0 precision  0.808511  0.808511  0.808511  0.808511
                    recall     0.090476  0.090476  0.090476  0.090476
                    f1         0.162741  0.162741  0.162741  0.162741

[72 rows x 4 columns]

In [99]:
def get_alg_data_met(df_met, data="alpha", met="f1"):
    df_met.index = df_met.index.set_names(["data", "algo", "budget", "metrics"])
    rdf = df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "metrics"], columns=["budget"]).loc[(data, slice(None), met)].droplevel(0, axis=1).droplevel([0, 2], axis=0)
    rdf.index = [f"sockfarm-{i}" for i in rdf.index]
    return rdf

In [107]:
get_alg_data_met(df_met, data="otc", met="recall")

budget,100.0,200.0,300.0,400.0
sockfarm-fraudar,0.433333,0.300000,0.229412,0.185714
sockfarm-rev2,0.211111,0.146154,0.111765,0.090476
sockfarm-rsd,0.294444,0.203846,0.155882,0.126190
